In [8]:
import fubon_neo
fubon_neo.__version__

# 匯入 SDK Library
from fubon_neo.sdk import FubonSDK, Order
from fubon_neo.constant import TimeInForce, OrderType, PriceType, MarketType, BSAction

# 連結 API Server
sdk = FubonSDK()

# 登入
accounts = sdk.login()   # 登入帳號 輸入:帳號、密碼、憑證路徑、憑證密碼
print(accounts)

# 建立行情元件連線
sdk.init_realtime() 

import json
import traceback

subscribe_ids = []  # 訂閱頻道 id 列表

def handle_connect():  # 連線成功 callback
    print("行情連接成功")

def handle_disconnect(code, message):  # 連接斷線 callback
    print(f"行情連接斷線: {code}, {message}")

def handle_message(message): # 處理接收訊息 callback
    try:
        msg = json.loads(message)
        event = msg["event"]
        data = msg["data"]
    
        if event == "pong":
            return
        
        if event == "subscribed":
            id = data["id"]
            
            if id in subscribe_ids:
                print(f"Error: 訂閱 id {id} 已存在列表中")
            else:
                subscribe_ids.append(id)
    
        elif event == "unsubscribed":
            id = data["id"]
            
            try:
                subscribe_ids.remove(id)
            except:
                print(f"Error: 查無此筆訂閱 id 資料, id {id}")
    
        print(f'market data message: {message}')           
        
    except Exception as e:
        handle_error(f'Error parsing JSON: {e}', traceback.format_exc())

def handle_error(error,traceback_info=None):  # 處理程式錯誤訊息 callback
    print(f'market data error: {error}')
    if traceback_info:
        print(f'Traceback:\n{traceback_info}')

stock = sdk.marketdata.websocket_client.stock
stock.on("connect", handle_connect)
stock.on("message", handle_message)
stock.on("disconnect", handle_disconnect)
stock.on("error", handle_error)

stock.connect()  # WebSocket 連線

stock_list = "3535"

# 建立行情查詢 WebAPI 連線 Object Instance
restStock = sdk.marketdata.rest_client.stock  

# 取得股票資訊 (依股票代碼查詢)
result = restStock.intraday.ticker(symbol=stock_list)
#print(result)

# 股票即時報價（依代碼查詢）
print("股票即時報價（依代碼查詢）")
result = restStock.intraday.quote(symbol=stock_list)
print(result)

# 股票價格Ｋ線（依代碼查詢）
print("股票價格Ｋ線（依代碼查詢）")
result = restStock.intraday.candles(symbol=stock_list, timeframe=5)
print(result)

Result {
  is_success: True,
  message: None,
  data: [Account {
    name: "張秉文",
    branch_no: "20117",
    account: "976793",
    account_type: "stock",
}]
}
行情連接成功
行情連接斷線: 1001, Maximum number of connections reached


Exception: authentication timeout